# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-c19f714ccc-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Club explorative search") 

Consider the following exploratory information need:

> investigate the association football clubs and find the main BGPs related to this sport. Compare clubs in countries and championships won. Explore also the participation of the countries in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football club

2. Identify the BGP for association football league

3. Return for each north american country the number of association football clubs

4. Identify the BGP for the UEFA Champions League.

5. Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

6. Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

7. Identify the BGP of the FIFA World Cup

8. Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

9. Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).


## Task 1
Identify the BGP for association football club

In [60]:
# query example

# Query 1

# Let check objects related to Cristiano Ronaldo
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?o .
   ?o wdt:P31 ?obj.
   
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?pName.
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('obj', 'http://www.wikidata.org/entity/Q3046190'), ('name', 'ESPY Award')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q3046190'), ('name', 'ESPY Award')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q615699'), ('name', 'Internet encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('obj', 'http://www.wikidata.org/entity/Q900341'), ('name', 'Laureus World Sports Awards')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q212434'), ('name', 'Olympic sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('p', 'http://

Final query for this task

In [14]:
# write your final query

# Query 2

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club

queryString = """
SELECT DISTINCT (wd:Q476028 as ?IRI) ?name
WHERE {

   # get the label
   wd:Q476028 sc:name ?name.
}

"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
1


## Task 2
Identify the BGP for association football league

In [ ]:
# write your queries
# It is found from Query 1 result.

Final query for this task

In [15]:
# write your final query

# Query 3

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league   (Query 1)

queryString = """
SELECT DISTINCT (wd:Q15991303 as ?IRI) ?name
WHERE {

   # get the label
   wd:Q15991303 sc:name ?name.
}

"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
1


## Task 3
Return for each north american country the number of association football clubs

In [47]:
# write your queries

# Query 4

# I should find information for 'country','continent', 'north american', and relation between country and association football club.

# From another workflow I know that
    # wdt:P17 => 'country'
    # wdt:P30 => 'continent'
    # wd:Q46 => 'Europe'
# However, I suppose every workflow as totally independent work. Thus I will try to find them again.

# Finding an association football club as sample.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country  (Query 1)

queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
    ?s wdt:P31 wd:Q476028 .
   
   # get the label
   ?s sc:name ?name.
}

LIMIT 1
"""
print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4647107'), ('name', 'A.D. Dili Leste')]
1


In [48]:
# Query 5

# I should find information for 'country','continent', 'north american', and relation between country and association football club.

# Finding relation between a football club and country.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country

# wd:Q4647107 => A.D. Dili Leste

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    wd:Q4647107 ?p ?o .
    ?o wdt:P31 wd:Q6256 .
   
   # get the label
   ?p sc:name ?name.
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
1


In [55]:
# Query 6

# Finding 'country','continent', 'north american', and relation between country and association football club.

# Finding 'Canada' as a north american country. 
# It is possible to find it without using 'FILTER'. In this case, the output will be list of all countries (limited to 200 because of LIMIT 200).
# I think filtering is a better choice.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country

# wd:Q4647107 => A.D. Dili Leste

queryString = """
SELECT DISTINCT ?c ?name
WHERE {
   # bind something
   ?c wdt:P31 wd:Q6256 .
   
   # get the label
   ?c sc:name ?name.
   
   FILTER(REGEX(?name,"[Cc]anada"))
}
ORDER BY ?name
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
1


In [56]:

# Query 7

# Finding 'country','continent' and 'north american'

# Looking for predicate that connects Canada to North America.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country

# wd:Q16 => Canada   --->(a north american country)

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q16 ?p ?o.
   
   # get the label
   ?o sc:name ?oName.
   ?p sc:name ?pName.
   
   FILTER(REGEX(?pName, "[Cc]ontinent") && REGEX(?oName, "[Nn]orth"))
}
ORDER BY ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pName', 'continent'), ('o', 'http://www.wikidata.org/entity/Q49'), ('oName', 'North America')]
1


In [60]:

# Query 8

# Task 3: Return for each north american country the number of association football clubs

# Listing all association football clubs for all north american countries.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America

queryString = """
SELECT DISTINCT ?sportClub ?country 
WHERE {
   # bind something
   ?s wdt:P31 wd:Q476028 ;
       wdt:P17 ?c .
   ?c  wdt:P30 wd:Q49 .
      
   
   # get the label
   ?s sc:name ?sportClub.
   ?c sc:name ?country
}
ORDER BY ?country
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sportClub', 'Grenades FC'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Parham FC'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Antigua Barracuda FC'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Hoppers F.C.'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'All Saints United F.C.'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Pigotts Bullets FC'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'SAP FC'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Empire F.C.'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Bassa Sports Club'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Sea View Farm FC'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Old Road FC'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Five Islands F.C.'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Tryum F.C.'), ('country', 'Antigua and Barbuda')]
[('sportClub', 'Rendezvous FC'), ('country', 'Barbados')]
[('sportClub', 'Notre Dame

Final query for this task

In [60]:
# write your final query

# Query 9

# Task 3: Return for each north american country the number of association football clubs

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America

queryString = """
SELECT DISTINCT (?c as ?IRI) ?country  (COUNT(DISTINCT(?s)) as ?noOfAssociationClubs)
WHERE {
   # bind something
   ?s wdt:P31 wd:Q476028 ;
       wdt:P17 ?c .
   ?c  wdt:P30 wd:Q49 .
      
   
   # get the label
   ?s sc:name ?sportClub.
   ?c sc:name ?country.
}
GROUP BY (?c) (?country)
ORDER BY ?country
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q781'), ('country', 'Antigua and Barbuda'), ('noOfAssociationClubs', '13')]
[('IRI', 'http://www.wikidata.org/entity/Q244'), ('country', 'Barbados'), ('noOfAssociationClubs', '10')]
[('IRI', 'http://www.wikidata.org/entity/Q242'), ('country', 'Belize'), ('noOfAssociationClubs', '35')]
[('IRI', 'http://www.wikidata.org/entity/Q16'), ('country', 'Canada'), ('noOfAssociationClubs', '136')]
[('IRI', 'http://www.wikidata.org/entity/Q800'), ('country', 'Costa Rica'), ('noOfAssociationClubs', '45')]
[('IRI', 'http://www.wikidata.org/entity/Q241'), ('country', 'Cuba'), ('noOfAssociationClubs', '19')]
[('IRI', 'http://www.wikidata.org/entity/Q35'), ('country', 'Denmark'), ('noOfAssociationClubs', '158')]
[('IRI', 'http://www.wikidata.org/entity/Q784'), ('country', 'Dominica'), ('noOfAssociationClubs', '7')]
[('IRI', 'http://www.wikidata.org/entity/Q786'), ('country', 'Dominican Republic'), ('noOfAssociationClubs', '13')]
[('IRI', 'http://www.wiki

## Task 4
Identify the BGP for the UEFA Champions League.

In [74]:
# write your queries

# Query 10

# Finding list of all continents
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)

queryString = """
SELECT DISTINCT ?c ?name
WHERE {
   # bind something
    ?s wdt:P31 wd:Q6256 ;
       wdt:P30 ?c .
   
   # get the label
   ?c sc:name ?name.
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('c', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('c', 'http://www.wikidata.org/entity/Q27611'), ('name', 'Central America')]
[('c', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('c', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('c', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('c', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('c', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
8


In [79]:

# Query 11

# Maybe 'edition of the UEFA European Championship' helps to find 'UEFA Champions League'.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)

queryString = """
SELECT DISTINCT ?p ?pName ?l ?name
WHERE {
   # bind something
    wd:Q107540719 ?p ?l .

   
   # get the label
   ?l sc:name ?name.
   ?p sc:name ?pName.
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('l', 'http://www.wikidata.org/entity/Q8960261'), ('name', 'Category:UEFA European Championship tournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('l', 'http://www.wikidata.org/entity/Q35572'), ('name', 'Union of European Football Associations')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('l', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('l', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
4


In [102]:
# Query 12

# Task 4: Identify the BGP for the UEFA Champions League.

# Checking subjects with organizer equal to UEFA for related information.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer


queryString = """
SELECT DISTINCT ?s ?sName 
WHERE {
   # bind something
     ?s wdt:P664 wd:Q35572.
   
   # get the label
   ?s sc:name ?sName.
   
   FILTER(REGEX(?sName,"UEFA.*[Cc]hampions"))
}
ORDER BY ?sName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q182201'), ('sName', '1968 UEFA European Championship')]
[('s', 'http://www.wikidata.org/entity/Q1443897'), ('sName', '1972 UEFA European Under-23 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q2465661'), ('sName', '1974 UEFA European Under-23 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q970564'), ('sName', '1976 UEFA European Under-23 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q1190899'), ('sName', '1978 UEFA European Under-21 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q956638'), ('sName', '1980 UEFA European Under-21 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q901668'), ('sName', '1982 UEFA European Under-16 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q602758'), ('sName', '1982 UEFA European Under-21 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q551493'), ('sName', '1984 UEFA European Under-16 Footba

In [110]:
# Query 13

# Task 4: Identify the BGP for the UEFA Champions League.

# Points from Query 12:
    # Almost got sure that 'UEFA Champions League' is the correct spell, thus I modified filtering.
    # It seems that year appears at the begining of the label. I am looking for general 'UEFA Champions League' without year.
    # Thus I sort the result DESC, supposing that nodes without any numbre at their beginning will be reported.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer


queryString = """
SELECT DISTINCT ?s ?sName 
WHERE {
   # bind something
     ?s wdt:P664 wd:Q35572.
   
   # get the label
   ?s sc:name ?sName.
   
   FILTER(REGEX(?sName,"UEFA Champions League"))
}
ORDER BY DESC(?sName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1453101'), ('sName', 'UEFA Champions League Anthem')]
[('s', 'http://www.wikidata.org/entity/Q189180'), ('sName', 'UEFA Champions League 2010-11')]
[('s', 'http://www.wikidata.org/entity/Q18756'), ('sName', 'UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q108662521'), ('sName', '2023–24 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q106086340'), ('sName', '2022–23 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q96621465'), ('sName', '2021–22 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q63088790'), ('sName', '2020–21 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q64547253'), ('sName', '2019–20 UEFA Champions League qualifying phase and play-off round')]
[('s', 'http://www.wikidata.org/entity/Q73798270'), ('sName', '2019–20 UEFA Champions League knockout phase')]
[('s', 'http://www.wikidata.org/entity/Q53164830'), ('sName', '2019–20 UEFA Champions Leag

Final query for this task

In [2]:
# write your final query

# Query 14

# Task 4: Identify the BGP for the UEFA Champions League.

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wd:Q476028 => association football club
# wd:Q15991303 => association football league
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League

queryString = """
SELECT DISTINCT (wd:Q18756 as ?IRI) ?name 
WHERE {
   
   # get the label
   wd:Q18756 sc:name ?name.
   
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)


Results
[('IRI', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
1


## Task 5
Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

In [74]:
# write your queries

# Query 15

# Queries 12 & 13 show that UEFA Champions League is held annually, thus I should find the winner of each year.
# XXXX UEFA Champions League (XXXX is the year) should have a relation to UEFA Champions League. Let's check it.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'


queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   
    wd:Q456097 ?p wd:Q18756 .
    
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
1


In [118]:
# Query 16

# Now I want to find how the winner is saved.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition


queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   
    wd:Q456097 ?p ?o .
    
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('o', 'http://www.wikidata.org/entity/Q53385703'), ('oName', '2000-2001 one-year-period')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q27020041'), ('oName', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team'), ('o', 'http://www.wikidata.org/entity/Q15789'), ('oName', 'FC Bayern Munich')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q15789'), ('oName', 'FC Bayern Munich')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition'), ('o', 'http://www.wikidata.org/entity/Q18756'), ('oName', 'UEFA Champions League')]
[('p

In [47]:
# Query 17

# Task 5: Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League.
# Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

# Finding winner of all seasons of UEFA Champions League.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner

queryString = """
SELECT DISTINCT ?sName ?tName  ?cName
WHERE {
   
    ?s wdt:P3450 wd:Q18756 ;
       wdt:P1346 ?t .
     ?t  wdt:P17 ?c .      
    
   # get the label
   ?s sc:name ?sName.
   ?t sc:name ?tName.
   ?c sc:name ?cName.
   
}
ORDER BY ASC(?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sName', '1992–93 UEFA Champions League'), ('tName', 'Olympique de Marseille'), ('cName', 'France')]
[('sName', '2000–01 UEFA Champions League'), ('tName', 'FC Bayern Munich'), ('cName', 'Germany')]
[('sName', '2012–13 UEFA Champions League'), ('tName', 'FC Bayern Munich'), ('cName', 'Germany')]
[('sName', '1975–76 European Cup'), ('tName', 'FC Bayern Munich'), ('cName', 'Germany')]
[('sName', '1973–74 European Cup'), ('tName', 'FC Bayern Munich'), ('cName', 'Germany')]
[('sName', '2019–20 UEFA Champions League'), ('tName', 'FC Bayern Munich'), ('cName', 'Germany')]
[('sName', '1974–75 European Cup'), ('tName', 'FC Bayern Munich'), ('cName', 'Germany')]
[('sName', '1996–97 UEFA Champions League'), ('tName', 'Borussia Dortmund'), ('cName', 'Germany')]
[('sName', '1982–83 European Cup'), ('tName', 'Hamburger SV'), ('cName', 'Germany')]
[('sName', '1984–85 European Cup'), ('tName', 'Juventus F.C.'), ('cName', 'Italy')]
[('sName', '1995–96 UEFA Champions League'), ('tName', 'Juve

Final query for this task

In [50]:
# write your final query

# Query 18

# Task 5: Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League.
# Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

# Query 17 shows that before 1991-92 'UEFA Champions League' was known as 'European Cup'. I do not exclude these years because they are related to UEFA Champions League by 'wdt:P3450'.
# In addition, 'Netherlands' appears with two different name ('Kingdom of the Netherlands' and 'Netherlands'). However, since it hase more than two different team with 'Netherlands' name, 
# it whould be in the result and no extra activity is needed due to different names.

# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner


queryString = """
SELECT DISTINCT ?c as ?IRI ?country (COUNT(DISTINCT(?t)) as ?clubs)
WHERE {
   
    ?s wdt:P3450 wd:Q18756 ;
       wdt:P1346 ?t . 
    ?t wdt:P17 ?c .      
    
   # get the label
   ?c sc:name ?country.
   
}
GROUP BY ?c ?country
HAVING (COUNT(DISTINCT(?t)) > 1)
ORDER BY (?country)
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('IRI', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('clubs', '3')]
[('IRI', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('clubs', '3')]
[('IRI', 'http://www.wikidata.org/entity/Q55'), ('country', 'Netherlands'), ('clubs', '3')]
[('IRI', 'http://www.wikidata.org/entity/Q45'), ('country', 'Portugal'), ('clubs', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q29'), ('country', 'Spain'), ('clubs', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q145'), ('country', 'United Kingdom'), ('clubs', '6')]
6


## Task 6
Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

In [110]:
# write your queries

# Query 19

# Cheking all leagues of european countries to have a clue.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner


queryString = """
SELECT DISTINCT ?cName ?s ?sName 
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c.
    ?c wdt:P30 wd:Q46.
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
   
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('cName', 'Albania'), ('s', 'http://www.wikidata.org/entity/Q1383952'), ('sName', 'Albanian Second Division')]
[('cName', 'Albania'), ('s', 'http://www.wikidata.org/entity/Q116909'), ('sName', 'Albanian First Division')]
[('cName', 'Albania'), ('s', 'http://www.wikidata.org/entity/Q1349507'), ('sName', 'Albanian Third Division')]
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q541065'), ('sName', 'Segona Divisió')]
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió')]
[('cName', 'Armenia'), ('s', 'http://www.wikidata.org/entity/Q2299810'), ('sName', 'Armenian First League')]
[('cName', 'Armenia'), ('s', 'http://www.wikidata.org/entity/Q237181'), ('sName', 'Armenian Premier League')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q683390'), ('sName', 'Austrian Regional League')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga')]
[('cName', 'Austria'

In [121]:

# Query 20

# Query 19 shows that it's not possible to find top league from league name. Let's investigate properties of different league.
# I will check Armania because it has just two leagues, thus it should be easier to find which one is top league.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner

# wd:Q2299810 => Armenian First League
# wd:Q237181 => Armenian Premier League

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   
    wd:Q2299810 ?p ?o .
    
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
ORDER BY (?pName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q399'), ('oName', 'Armenia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15991303'), ('oName', 'association football league')]
[('p', 'http://www.wikidata.org/prop/direct/P2499'), ('pName', 'league level above'), ('o', 'http://www.wikidata.org/entity/Q237181'), ('oName', 'Armenian Premier League')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q391151'), ('oName', 'Football Federation of Armenia')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q110'), ('oName', 'March')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oName', 'association football')]
[('p', 'http://www.wikidata.org/prop/

In [84]:

# Query 21

# Query 19 shows that it's not possible to find top league from league name. Let's investigate properties of different league.
# I will check Armania because it has just two leagues, thus it should be easier to find which one is top league.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner

# wd:Q2299810 => Armenian First League
# wd:Q237181 => Armenian Premier League

queryString = """
SELECT DISTINCT  ?p ?pName ?o ?oName
WHERE {
   
    wd:Q237181 ?p ?o .
    
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
ORDER BY (?pName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q31930761'), ('oName', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q399'), ('oName', 'Armenia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15991303'), ('oName', 'association football league')]
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('pName', 'league level below'), ('o', 'http://www.wikidata.org/entity/Q2299810'), ('oName', 'Armenian First League')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q391151'), ('oName', 'Football Federation of Armenia')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q118'), ('oName', 'April')]
[('p', 'http://

In [108]:

# Query 22

# Queries 20 & 21 show that by 'league level below (wdt:P2500)' and 'league level above (wdt:P2499)' it is possible to find top level league.
# Top level league is the league that does not have 'league level above (wdt:P2499)'.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below

# wd:Q2299810 => Armenian First League
# wd:Q237181 => Armenian Premier League


queryString = """
SELECT DISTINCT ?cName ?s ?sName
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c.
    ?c wdt:P30 wd:Q46.
    
    FILTER NOT EXISTS {?s wdt:P2499 ?o.}
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
   
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió')]
[('cName', 'Armenia'), ('s', 'http://www.wikidata.org/entity/Q237181'), ('sName', 'Armenian Premier League')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q4825490'), ('sName', 'Austrian Regional League Central')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q4825491'), ('sName', 'Austrian Regional League East')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q687207'), ('sName', 'ÖFB-Frauenliga')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q2436235'), ('sName', 'UPC Tirol Liga')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q33113671'), ('sName', 'Kärntner Liga')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q4825492'), ('sName', 'Austrian Regional League West')]
[('cName', 'Aus

In [93]:

# Query 23

# Filtering with P2499 is not enough. 
# I am going to check properties of other leagues to find a solution.
# Austria has some leagues that seems are local such as 'Austrian Regional League Central', and others that I do not have any idea about them.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league (Query 1)
# wd:Q15991303 => association football league (Query 1)
# wd:Q107540719 => edition of the UEFA European Championship (Query 1)
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below

# wd:Q219592 => Austrian Bundesliga
# wd:Q4825490 => Austrian Regional League Central


queryString = """
SELECT DISTINCT  ?p ?pName ?o ?oName
WHERE {
   
    wd:Q219592 ?p ?o .
    
   # get the label
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
ORDER BY (?pName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q31930761'), ('oName', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q40'), ('oName', 'Austria')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15991303'), ('oName', 'association football league')]
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('pName', 'league level below'), ('o', 'http://www.wikidata.org/entity/Q650236'), ('oName', 'Austrian Football Second League')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q4009944'), ('oName', 'Österreichische Fußball-Bundesliga')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q121'), ('oName', 'July')]
[('

In [112]:
# Query 24

# Now I check not existance of P2499, existance of P2500 and being subclass of 'national championship (Q3270632)'.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship


queryString = """
SELECT DISTINCT ?cName ?s ?sName 
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c ;
       wdt:P279 wd:Q3270632 .
    ?c wdt:P30 wd:Q46.
    
    FILTER NOT EXISTS {?s wdt:P2499 ?o.}
    FILTER EXISTS {?s wdt:P2500 ?o.}
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
   
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga')]
[('cName', 'Azerbaijan'), ('s', 'http://www.wikidata.org/entity/Q326859'), ('sName', 'Azerbaijan Premier League')]
[('cName', 'Belarus'), ('s', 'http://www.wikidata.org/entity/Q244464'), ('sName', 'Belarusian Premier League')]
[('cName', 'Belgium'), ('s', 'http://www.wikidata.org/entity/Q216022'), ('sName', 'Belgian First Division A')]
[('cName', 'Bosnia and Herzegovina'), ('s', 'http://www.wikidata.org/entity/Q391006'), ('sName', 'Premier League of Bosnia and Herzegovina')]
[('cName', 'Bulgaria'), ('s', 'http://www.wikidata.org/entity/Q12837728'), ('sName', 'First Professional Football League')]
[('cName', 'Czech Republic'), ('s', 'http://www.wikidata.org/entity/Q217016'), ('sName', 'Czech First League')]
[('cName', 'Denmark'), ('s', 'http://www.wikidata.org/entity/Q739479

In [2]:
# Query 25

# It seems that Query 24 is the answer to task 6, but still for some countries I found more than one league as the top level league.
# Also it returns league for 'German Democratic Republic'. This country is not exists anymore.
# Let's check all properties of a league for a country that has more than one league in the result.

# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship

# wd:Q220875 => eikkausliiga     -->(Finland)
# wd:Q1429268 => Kansallinen Liiga  -->(Finland)

queryString = """
SELECT DISTINCT  ?p ?pName 
WHERE {
   
     wd:Q220875 ?p ?o.
    
   # get the label
   ?p sc:name ?pName.
   #?o sc:name ?oName.
}
ORDER BY (?pName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pName', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6262'), ('pName', 'Fandom article ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('pName', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P7441'), ('pName', 'WorldFootball.net competition ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6293'), ('pName', 'YSA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('pName', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8309'), ('pName', 'Yle topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http

In [148]:
# Query 26

# I will use 'sports league level (P3983)' to check what it means. 
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship
# wdt:P3983 => sports league level

queryString = """
SELECT DISTINCT ?cName ?s ?sName ?sll
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c ;
       wdt:P3983 ?sll ;
       wdt:P279 wd:Q3270632 .
    ?c wdt:P30 wd:Q46.
    
    FILTER NOT EXISTS {?s wdt:P2499 ?o.}
    FILTER EXISTS {?s wdt:P2500 ?o.}
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió'), ('sll', '1')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga'), ('sll', '1')]
[('cName', 'Azerbaijan'), ('s', 'http://www.wikidata.org/entity/Q326859'), ('sName', 'Azerbaijan Premier League'), ('sll', '1')]
[('cName', 'Belarus'), ('s', 'http://www.wikidata.org/entity/Q244464'), ('sName', 'Belarusian Premier League'), ('sll', '1')]
[('cName', 'Belgium'), ('s', 'http://www.wikidata.org/entity/Q216022'), ('sName', 'Belgian First Division A'), ('sll', '1')]
[('cName', 'Bosnia and Herzegovina'), ('s', 'http://www.wikidata.org/entity/Q391006'), ('sName', 'Premier League of Bosnia and Herzegovina'), ('sll', '1')]
[('cName', 'Bulgaria'), ('s', 'http://www.wikidata.org/entity/Q12837728'), ('sName', 'First Professional Football League'), ('sll', '1')]
[('cName', 'Czech Republic'), ('s', 'http://www.wikidata.org/entity/Q217016'), ('s

In [147]:
# Query 27

# 'sports league level' for all leagues in Query 26 are one. Now I am checking it before filtering, so I can see what is the sll for leagues that are filtered out.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship
# wdt:P3983 => sports league level

queryString = """
SELECT DISTINCT ?cName ?s ?sName ?sll
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c ;
       wdt:P3983 ?sll .
    #   wdt:P279 wd:Q3270632 .
    ?c wdt:P30 wd:Q46.
    
    #FILTER NOT EXISTS {?s wdt:P2499 ?o.}
    #FILTER EXISTS {?s wdt:P2500 ?o.}
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió'), ('sll', '1')]
[('cName', 'Armenia'), ('s', 'http://www.wikidata.org/entity/Q237181'), ('sName', 'Armenian Premier League'), ('sll', '1')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga'), ('sll', '1')]
[('cName', 'Azerbaijan'), ('s', 'http://www.wikidata.org/entity/Q326859'), ('sName', 'Azerbaijan Premier League'), ('sll', '1')]
[('cName', 'Belarus'), ('s', 'http://www.wikidata.org/entity/Q244464'), ('sName', 'Belarusian Premier League'), ('sll', '1')]
[('cName', 'Belgium'), ('s', 'http://www.wikidata.org/entity/Q216022'), ('sName', 'Belgian First Division A'), ('sll', '1')]
[('cName', 'Bosnia and Herzegovina'), ('s', 'http://www.wikidata.org/entity/Q391006'), ('sName', 'Premier League of Bosnia and Herzegovina'), ('sll', '1')]
[('cName', 'Bulgaria'), ('s', 'http://www.wikidata.org/entity/Q12837728'), ('sName', 'First Prof

In [149]:
# Query 28

# Query 27 proofs that the result of Query 26 is correct and task is done.
# But maybe it is possible to complete the task just by using 'sports league level'.
# if it work it would be more efficient to use it. Now I am checking it.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship
# wdt:P3983 => sports league level

queryString = """
SELECT DISTINCT ?cName ?s ?sName 
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c ;
       wdt:P3983 1 .
    ?c wdt:P30 wd:Q46.
    
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió')]
[('cName', 'Armenia'), ('s', 'http://www.wikidata.org/entity/Q237181'), ('sName', 'Armenian Premier League')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga')]
[('cName', 'Azerbaijan'), ('s', 'http://www.wikidata.org/entity/Q326859'), ('sName', 'Azerbaijan Premier League')]
[('cName', 'Belarus'), ('s', 'http://www.wikidata.org/entity/Q244464'), ('sName', 'Belarusian Premier League')]
[('cName', 'Belgium'), ('s', 'http://www.wikidata.org/entity/Q216022'), ('sName', 'Belgian First Division A')]
[('cName', 'Bosnia and Herzegovina'), ('s', 'http://www.wikidata.org/entity/Q391006'), ('sName', 'Premier League of Bosnia and Herzegovina')]
[('cName', 'Bulgaria'), ('s', 'http://www.wikidata.org/entity/Q12837728'), ('sName', 'First Professional Football League')]
[('cName', 'Cyprus'), ('s', 'http://www.wikidata.org/entity/Q155965'),

In [152]:
# Query 29

# Query 28 works, but it returns leagues for countries that not exsits. 
# I am going to find a way to exclude them.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship
# wdt:P3983 => sports league level

# wd:Q157553 => DDR-Oberliga --> German Democratic Republic

queryString = """
SELECT DISTINCT ?p ?pName ?o
WHERE {
   
      wd:Q157553 ?p ?o  .

   # get the label
   ?p sc:name ?pName.

}
ORDER BY ?pName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID'), ('o', '03612291n')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('o', 'DDR-Oberliga')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('o', '/m/037kq2')]
[('p', 'http://www.wikidata.org/prop/direct/P7867'), ('pName', 'category for maps'), ('o', 'http://www.wikidata.org/entity/Q84022174')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q31930761')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q16957')]
[('p', 'http://www.wikidata.org/prop/direct/P576'), ('pName', 'dissolved, abolished or demolished date'), ('o', '1991-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('pName', 'event interval'), ('o', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'in

In [3]:
# Query 30

# Query 28 works, but it returns leagues for countries that are not exsits. 
# I remove all results with 'dissolved, abolished or demolished date' (P576), because it means that league is not exist now.

# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship
# wdt:P3983 => sports league level
# wdt:P576 => dissolved, abolished or demolished date

queryString = """
SELECT DISTINCT ?cName ?s ?sName 
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c ;
       wdt:P3983 1 .
    ?c wdt:P30 wd:Q46.
    
    FILTER NOT EXISTS {?s wdt:P576 ?o}
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió')]
[('cName', 'Armenia'), ('s', 'http://www.wikidata.org/entity/Q237181'), ('sName', 'Armenian Premier League')]
[('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga')]
[('cName', 'Azerbaijan'), ('s', 'http://www.wikidata.org/entity/Q326859'), ('sName', 'Azerbaijan Premier League')]
[('cName', 'Belarus'), ('s', 'http://www.wikidata.org/entity/Q244464'), ('sName', 'Belarusian Premier League')]
[('cName', 'Belgium'), ('s', 'http://www.wikidata.org/entity/Q216022'), ('sName', 'Belgian First Division A')]
[('cName', 'Bosnia and Herzegovina'), ('s', 'http://www.wikidata.org/entity/Q391006'), ('sName', 'Premier League of Bosnia and Herzegovina')]
[('cName', 'Bulgaria'), ('s', 'http://www.wikidata.org/entity/Q12837728'), ('sName', 'First Professional Football League')]
[('cName', 'Cyprus'), ('s', 'http://www.wikidata.org/entity/Q155965'),

Final query for this task

In [155]:
# write your final query

# Query 31

# Task 6: Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

# I can add  "FILTER NOT EXISTS {?s wdt:P576 ?o}" to Query 24 and get the same result as Query 30.
# In this case, Query 24 will have 3 filtering statements while Query 30 has one.
# Thus I think Query 30 is more efficient and I selected it as final query.
# Sorting the result is just for make it easier to review.

# wd:Q476028 => association football club
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P30 => continent
# wd:Q49 => North America
# wd:Q46 => Europe
# wdt:P118 => league
# wd:Q15991303 => association football league
# wd:Q107540719 => edition of the UEFA European Championship
# wd:Q35572 => Union of European Football Associations
# wdt:P664 => organizer
# wd:Q18756 => UEFA Champions League
# Q456097 => '2000–01 UEFA Champions League'
# wdt:P3450 => sports season of league or competition
# wdt:P1346 => winner
# wdt:P2499 => league level above
# wdt:P2500 => league level below
# wd:Q3270632 => national championship
# wdt:P3983 => sports league level
# wdt:P576 => dissolved, abolished or demolished date

queryString = """
SELECT DISTINCT ?c ?cName ?s ?sName 
WHERE {
   
    ?s wdt:P31 wd:Q15991303 ;
       wdt:P17 ?c ;
       wdt:P3983 1 .
    ?c wdt:P30 wd:Q46.
    
    FILTER NOT EXISTS {?s wdt:P576 ?o}
    
   # get the label
   ?s sc:name ?sName.
   ?c sc:name ?cName.
}
ORDER BY (?cName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('s', 'http://www.wikidata.org/entity/Q257282'), ('sName', 'Primera Divisió')]
[('c', 'http://www.wikidata.org/entity/Q399'), ('cName', 'Armenia'), ('s', 'http://www.wikidata.org/entity/Q237181'), ('sName', 'Armenian Premier League')]
[('c', 'http://www.wikidata.org/entity/Q40'), ('cName', 'Austria'), ('s', 'http://www.wikidata.org/entity/Q219592'), ('sName', 'Austrian Bundesliga')]
[('c', 'http://www.wikidata.org/entity/Q227'), ('cName', 'Azerbaijan'), ('s', 'http://www.wikidata.org/entity/Q326859'), ('sName', 'Azerbaijan Premier League')]
[('c', 'http://www.wikidata.org/entity/Q184'), ('cName', 'Belarus'), ('s', 'http://www.wikidata.org/entity/Q244464'), ('sName', 'Belarusian Premier League')]
[('c', 'http://www.wikidata.org/entity/Q31'), ('cName', 'Belgium'), ('s', 'http://www.wikidata.org/entity/Q216022'), ('sName', 'Belgian First Division A')]
[('c', 'http://www.wikidata.org/entity/Q225'), ('cName', 'Bosn

## Task 7
Identify the BGP of the FIFA World Cup

In [163]:
# write your queries

# Query 32

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1)
# wd:Q27020041 => sports season  (Query 1)

queryString = """
SELECT DISTINCT ?o ?oName 
WHERE {
   
    wd:Q11571 wdt:P1344 ?o  .

   # get the label
   ?o sc:name ?oName.
}
ORDER BY (?oName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q8558'), ('oName', '2004 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q37285'), ('oName', '2006 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q176883'), ('oName', '2010 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q79859'), ('oName', '2014 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q170645'), ('oName', '2018 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q324867'), ('oName', 'La Liga')]
[('o', 'http://www.wikidata.org/entity/Q102920'), ('oName', 'UEFA Euro 2004')]
[('o', 'http://www.wikidata.org/entity/Q241864'), ('oName', 'UEFA Euro 2008')]
[('o', 'http://www.wikidata.org/entity/Q22669'), ('oName', 'UEFA Euro 2012')]
[('o', 'http://www.wikidata.org/entity/Q189571'), ('oName', 'UEFA Euro 2016')]
[('o', 'http://www.wikidata.org/entity/Q373501'), ('oName', 'UEFA Euro 2020')]
11


In [173]:
# Query 33

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in 
# wd:Q27020041 => sports season 
# wd:Q170645 => 2018 FIFA World Cup  (Query 32)

queryString = """
SELECT DISTINCT ?o ?oName 
WHERE {
   
    wd:Q170645  ?p ?o.

   # get the label
   ?o sc:name ?oName.

}
ORDER BY (?oName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q31043671'), ('oName', '2018 FIFA World Cup Final')]
[('o', 'http://www.wikidata.org/entity/Q31189405'), ('oName', '2018 FIFA World Cup Group A')]
[('o', 'http://www.wikidata.org/entity/Q31189406'), ('oName', '2018 FIFA World Cup Group B')]
[('o', 'http://www.wikidata.org/entity/Q31189411'), ('oName', '2018 FIFA World Cup Group C')]
[('o', 'http://www.wikidata.org/entity/Q31189412'), ('oName', '2018 FIFA World Cup Group D')]
[('o', 'http://www.wikidata.org/entity/Q31189413'), ('oName', '2018 FIFA World Cup Group E')]
[('o', 'http://www.wikidata.org/entity/Q43214797'), ('oName', '2018 FIFA World Cup Group F')]
[('o', 'http://www.wikidata.org/entity/Q43214817'), ('oName', '2018 FIFA World Cup Group G')]
[('o', 'http://www.wikidata.org/entity/Q43214843'), ('oName', '2018 FIFA World Cup Group H')]
[('o', 'http://www.wikidata.org/entity/Q43214603'), ('oName', '2018 FIFA World Cup knockout stage')]
[('o', 'http://www.wikidata.org/entity/Q5496873

Final query for this task

In [30]:
# write your final query

# Query 34

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in 
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup

queryString = """
SELECT DISTINCT (wd:Q19317 as ?IRI) ?Name 
WHERE {

   # get the label
   wd:Q19317 sc:name ?Name.

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q19317'), ('Name', 'FIFA World Cup')]
1


## Task 8
Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

In [42]:
# write your queries

# Query 35

# At first I want to get some general information about Fifa world cup. 
# Related usefull node/properties till now are as below.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1)
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT ?t ?tName 
WHERE {
    ?t  wdt:P1344 wd:Q170645 .
    
   # get the label
   ?t sc:name ?tName.

}
ORDER BY ?tName
LIMIT 50
"""

print("Results")
x=run_query(queryString)



Results
[('t', 'http://www.wikidata.org/entity/Q54812340'), ('tName', '2018 FIFA World Cup team')]
[('t', 'http://www.wikidata.org/entity/Q478941'), ('tName', 'Aaron Mooy')]
[('t', 'http://www.wikidata.org/entity/Q2688057'), ('tName', 'Abdallah Said')]
[('t', 'http://www.wikidata.org/entity/Q20675902'), ('tName', 'Abdiel Arroyo')]
[('t', 'http://www.wikidata.org/entity/Q178493'), ('tName', 'Abdoulaye Diallo')]
[('t', 'http://www.wikidata.org/entity/Q28716544'), ('tName', 'Abdullah Al-Khaibari')]
[('t', 'http://www.wikidata.org/entity/Q22245519'), ('tName', 'Abdullah Al-Mayouf')]
[('t', 'http://www.wikidata.org/entity/Q10557884'), ('tName', 'Abdullah Otayf')]
[('t', 'http://www.wikidata.org/entity/Q16218584'), ('tName', 'Abdulmalek Al-Khaibri')]
[('t', 'http://www.wikidata.org/entity/Q318416'), ('tName', 'Abel Aguilar')]
[('t', 'http://www.wikidata.org/entity/Q26932598'), ('tName', 'Achraf Hakimi')]
[('t', 'http://www.wikidata.org/entity/Q4680085'), ('tName', 'Adama Mbengue')]
[('t', 'h

In [44]:

# Query 36

# At first I want to get some general information about Fifa world cup. 
# Related usefull node/properties till now are as below.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT ?t ?tName
WHERE {
    wd:Q170645 wdt:P1923 ?t.
    
   # get the label
   ?t sc:name ?tName.
}
ORDER BY ?tName
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('t', 'http://www.wikidata.org/entity/Q43196530'), ('tName', 'Argentina at the 2018 FIFA World Cup')]
[('t', 'http://www.wikidata.org/entity/Q45101586'), ('tName', 'Australia at the 2018 FIFA World Cup')]
[('t', 'http://www.wikidata.org/entity/Q43122121'), ('tName', 'Belgium at the 2018 FIFA World Cup')]
[('t', 'http://www.wikidata.org/entity/Q43196538'), ('tName', 'Brazil at the 2018 FIFA World Cup\u200e')]
[('t', 'http://www.wikidata.org/entity/Q43196480'), ('tName', 'Colombia at the 2018 FIFA World Cup')]
[('t', 'http://www.wikidata.org/entity/Q46265248'), ('tName', 'Costa Rica at the 2018 FIFA World Cup')]
[('t', 'http://www.wikidata.org/entity/Q45905387'), ('tName', 'Croatia at the 2018 FIFA World Cup')]
[('t', 'http://www.wikidata.org/entity/Q45186603'), ('tName', 'Denmark at the 2018 FIFA World Cup')]
[('t', 'http://www.wikidata.org/entity/Q44959733'), ('tName', 'Egypt at the 2018 FIFA World Cup\u200e')]
[('t', 'http://www.wikidata.org/entity/Q42944467'), ('tName', 'Eng

In [53]:
# Query 37

# At first I want to get some general information about Fifa world cup. 
# Related usefull node/properties till now are as below.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT ?c ?cName ?t ?tName
WHERE {
    wd:Q170645 wdt:P1923 ?t.
    ?t wdt:P17 ?c .
    ?c wdt:P31 wd:Q6256.
    
   # get the label
   ?t sc:name ?tName.
   ?c sc:name ?cName.

}
ORDER BY ?tName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q414'), ('cName', 'Argentina'), ('t', 'http://www.wikidata.org/entity/Q43196530'), ('tName', 'Argentina at the 2018 FIFA World Cup')]
[('c', 'http://www.wikidata.org/entity/Q408'), ('cName', 'Australia'), ('t', 'http://www.wikidata.org/entity/Q45101586'), ('tName', 'Australia at the 2018 FIFA World Cup')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('cName', 'Brazil'), ('t', 'http://www.wikidata.org/entity/Q43196538'), ('tName', 'Brazil at the 2018 FIFA World Cup\u200e')]
[('c', 'http://www.wikidata.org/entity/Q739'), ('cName', 'Colombia'), ('t', 'http://www.wikidata.org/entity/Q43196480'), ('tName', 'Colombia at the 2018 FIFA World Cup')]
[('c', 'http://www.wikidata.org/entity/Q800'), ('cName', 'Costa Rica'), ('t', 'http://www.wikidata.org/entity/Q46265248'), ('tName', 'Costa Rica at the 2018 FIFA World Cup')]
[('c', 'http://www.wikidata.org/entity/Q79'), ('cName', 'Egypt'), ('t', 'http://www.wikidata.org/entity/Q44959733'), ('tName', 

In [48]:
# Query 38

# Amonge 32 teams participated in Fifa world cup, just 14 of the have property of country (P17) ??????? Why?
# What is obtained from Queries 36-37 results:
    # Tunisia is related to 2018 FIFA World Cup (Q170645) by P279, not P31.
# Let's check teams of all FIFA world cups.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT  ?cup ?cupName (COUNT(DISTINCT(?t)) as ?teams)
WHERE {
    ?cup wdt:P3450 wd:Q19317 ;
         wdt:P1923 ?t.
    
   # get the label
   ?cup sc:name ?cupName.

}
GROUP BY ?cup ?cupName
ORDER BY ?cupName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('cup', 'http://www.wikidata.org/entity/Q48432'), ('cupName', '1930 FIFA World Cup'), ('teams', '13')]
[('cup', 'http://www.wikidata.org/entity/Q1477177'), ('cupName', '1934 FIFA World Cup'), ('teams', '16')]
[('cup', 'http://www.wikidata.org/entity/Q131591'), ('cupName', '1938 FIFA World Cup'), ('teams', '15')]
[('cup', 'http://www.wikidata.org/entity/Q132515'), ('cupName', '1950 FIFA World Cup'), ('teams', '13')]
[('cup', 'http://www.wikidata.org/entity/Q131075'), ('cupName', '1954 FIFA World Cup'), ('teams', '16')]
[('cup', 'http://www.wikidata.org/entity/Q132533'), ('cupName', '1958 FIFA World Cup'), ('teams', '16')]
[('cup', 'http://www.wikidata.org/entity/Q160813'), ('cupName', '1962 FIFA World Cup'), ('teams', '16')]
[('cup', 'http://www.wikidata.org/entity/Q134202'), ('cupName', '1966 FIFA World Cup'), ('teams', '16')]
[('cup', 'http://www.wikidata.org/entity/Q132664'), ('cupName', '1970 FIFA World Cup'), ('teams', '16')]
[('cup', 'http://www.wikidata.org/entity/Q16612

In [60]:
# Query 39

# Except first 4 cups, others seem correct. However, searching WEB shows that for the 4 fisrt cups (1930-1950) number of teams that is founded is correct.
# Now I am checking their countries.
# What is obtained from Queries 36-37 results:
    # Tunisia is related to 2018 FIFA World Cup (Q170645) by P279, not P31.
# Let's check teams of all FIFA world cups.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT  ?cup ?cupName (COUNT(DISTINCT(?c)) as ?countries)
WHERE {
    ?cup wdt:P3450 wd:Q19317 ;
         wdt:P1923 ?t.
    ?t   wdt:P17 ?c.    
   # get the label
   ?cup sc:name ?cupName.

}
GROUP BY ?cup ?cupName
ORDER BY ?cupName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('cup', 'http://www.wikidata.org/entity/Q48432'), ('cupName', '1930 FIFA World Cup'), ('countries', '13')]
[('cup', 'http://www.wikidata.org/entity/Q1477177'), ('cupName', '1934 FIFA World Cup'), ('countries', '16')]
[('cup', 'http://www.wikidata.org/entity/Q131591'), ('cupName', '1938 FIFA World Cup'), ('countries', '14')]
[('cup', 'http://www.wikidata.org/entity/Q132515'), ('cupName', '1950 FIFA World Cup'), ('countries', '13')]
[('cup', 'http://www.wikidata.org/entity/Q131075'), ('cupName', '1954 FIFA World Cup'), ('countries', '15')]
[('cup', 'http://www.wikidata.org/entity/Q132533'), ('cupName', '1958 FIFA World Cup'), ('countries', '13')]
[('cup', 'http://www.wikidata.org/entity/Q160813'), ('cupName', '1962 FIFA World Cup'), ('countries', '16')]
[('cup', 'http://www.wikidata.org/entity/Q134202'), ('cupName', '1966 FIFA World Cup'), ('countries', '16')]
[('cup', 'http://www.wikidata.org/entity/Q132664'), ('cupName', '1970 FIFA World Cup'), ('countries', '16')]
[('cup', 'h

In [5]:
# Query 40

# Comparing output of Query 39 and Query 40 leads to the result that for some teams the country property (P17) does not exist.
# To get sure I just check one of them. 'Belgium at the 2018 FIFA World Cup' (Q43122121) does not appear in the output of Query 37 but it is in the output ofQuery 36.
# What is obtained from Queries 36-37 results:
    # Tunisia is related to 2018 FIFA World Cup (Q170645) by P279, not P31.
# Let's check teams of all FIFA world cups.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT  ?p ?pName
WHERE {
    wd:Q43122121 ?p ?o.
    
   # get the label
   ?p sc:name ?pName.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('pName', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('pName', 'ranking')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P286'), ('pName', 'head coach')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pName', 'point in time')]
[('p', 'http://www.wikidata.org/

In [66]:
# Query 41

# Maybe 'country of sport' (P1532) is more suitable choice for finding countries.
# What is obtained from Queries 36-37 results:
    # Tunisia is related to 2018 FIFA World Cup (Q170645) by P279, not P31.
# Let's check teams of all FIFA world cups.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT  ?cup ?cupName (COUNT(DISTINCT(?c)) as ?countries)
WHERE {
    ?cup wdt:P3450 wd:Q19317 ;
         wdt:P1923 ?t.
    ?t   wdt:P1532 ?c.
    
   # get the label
   ?cup sc:name ?cupName.

}
GROUP BY ?cup ?cupName
ORDER BY ?cupName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('cup', 'http://www.wikidata.org/entity/Q48432'), ('cupName', '1930 FIFA World Cup'), ('countries', '12')]
[('cup', 'http://www.wikidata.org/entity/Q1477177'), ('cupName', '1934 FIFA World Cup'), ('countries', '16')]
[('cup', 'http://www.wikidata.org/entity/Q131591'), ('cupName', '1938 FIFA World Cup'), ('countries', '15')]
[('cup', 'http://www.wikidata.org/entity/Q132515'), ('cupName', '1950 FIFA World Cup'), ('countries', '12')]
[('cup', 'http://www.wikidata.org/entity/Q131075'), ('cupName', '1954 FIFA World Cup'), ('countries', '15')]
[('cup', 'http://www.wikidata.org/entity/Q132533'), ('cupName', '1958 FIFA World Cup'), ('countries', '15')]
[('cup', 'http://www.wikidata.org/entity/Q160813'), ('cupName', '1962 FIFA World Cup'), ('countries', '15')]
[('cup', 'http://www.wikidata.org/entity/Q134202'), ('cupName', '1966 FIFA World Cup'), ('countries', '16')]
[('cup', 'http://www.wikidata.org/entity/Q132664'), ('cupName', '1970 FIFA World Cup'), ('countries', '16')]
[('cup', 'h

In [6]:
# Query 42

# P1532 is better but still not complete. Let's check both of them (P1532 or P17).
# What is obtained from Queries 36-37 results:
    # Tunisia is related to 2018 FIFA World Cup (Q170645) by P279, not P31.
# Let's check teams of all FIFA world cups.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT  ?cup ?cupName (COUNT(DISTINCT(?c)) as ?countries)
WHERE {
    ?cup wdt:P3450 wd:Q19317 ;
         wdt:P1923 ?t.
    ?t   wdt:P1532|wdt:P17 ?c.
    
   # get the label
   ?cup sc:name ?cupName.

}
GROUP BY ?cup ?cupName
ORDER BY ?cupName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('cup', 'http://www.wikidata.org/entity/Q48432'), ('cupName', '1930 FIFA World Cup'), ('countries', '13')]
[('cup', 'http://www.wikidata.org/entity/Q1477177'), ('cupName', '1934 FIFA World Cup'), ('countries', '17')]
[('cup', 'http://www.wikidata.org/entity/Q131591'), ('cupName', '1938 FIFA World Cup'), ('countries', '15')]
[('cup', 'http://www.wikidata.org/entity/Q132515'), ('cupName', '1950 FIFA World Cup'), ('countries', '14')]
[('cup', 'http://www.wikidata.org/entity/Q131075'), ('cupName', '1954 FIFA World Cup'), ('countries', '17')]
[('cup', 'http://www.wikidata.org/entity/Q132533'), ('cupName', '1958 FIFA World Cup'), ('countries', '17')]
[('cup', 'http://www.wikidata.org/entity/Q160813'), ('cupName', '1962 FIFA World Cup'), ('countries', '17')]
[('cup', 'http://www.wikidata.org/entity/Q134202'), ('cupName', '1966 FIFA World Cup'), ('countries', '17')]
[('cup', 'http://www.wikidata.org/entity/Q132664'), ('cupName', '1970 FIFA World Cup'), ('countries', '17')]
[('cup', 'h

In [16]:
# Query 43

# Query 42 shows that one country has both P17 and P1532 properties, thus number of countries is more than number of teams. (compare with Query 38). Let's find it.
# What is obtained from Queries 36-37 results:
    # Tunisia is related to 2018 FIFA World Cup (Q170645) by P279, not P31.
# Let's check teams of all FIFA world cups.
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)

# wd:Q170645 => 2018 FIFA World Cup (Query 32)


queryString = """
SELECT DISTINCT ?tName ?c ?cName 
WHERE {

    wd:Q170645 wdt:P1923 ?t.
    ?t wdt:P1532|wdt:P17 ?c .
    
   # get the label
    ?t sc:name ?tName.
    ?c sc:name ?cName.

}
ORDER BY ?tName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('tName', 'Argentina at the 2018 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q414'), ('cName', 'Argentina')]
[('tName', 'Australia at the 2018 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q408'), ('cName', 'Australia')]
[('tName', 'Belgium at the 2018 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q31'), ('cName', 'Belgium')]
[('tName', 'Brazil at the 2018 FIFA World Cup\u200e'), ('c', 'http://www.wikidata.org/entity/Q155'), ('cName', 'Brazil')]
[('tName', 'Colombia at the 2018 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q739'), ('cName', 'Colombia')]
[('tName', 'Costa Rica at the 2018 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q800'), ('cName', 'Costa Rica')]
[('tName', 'Croatia at the 2018 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q224'), ('cName', 'Croatia')]
[('tName', 'Denmark at the 2018 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q35'), ('cName', 'Denmark')]
[('tName', 'Egypt at the 2018 

In [7]:
# Query 44

# task 8: Consider countries who participated at the FIFA World Cup more than 10 times.
#         Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

# 'England at the 2018 FIFA World Cup' appears twice in the result, with:
    # wd:Q145 => 'United Kingdom' 
    # wd:Q21 => 'England'

### Considering all obtained results by queries related to task 8, and the fact that the data source is a snapshot of wikidata with probable inconsistencies, I decided to use P1532 for finding countries.
### Although the result would not be very precise, I guess that the goal of the task is not finding and solving all inconsistencies, thus I make such a decision.
### However, If the only inconsistency is about England and United Kingdom as described above, just the number of apparitions for England will be incorrect.
    
# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)

queryString = """
SELECT DISTINCT  ?c ?country (COUNT(DISTINCT(?cup)) as ?apparitions)
WHERE {
    ?cup wdt:P3450 wd:Q19317 ;
         wdt:P1923 ?t.
    ?t   wdt:P1532 ?c.
    
   # get the label
   ?c sc:name ?country.

}
GROUP BY ?c ?country
Having (COUNT(DISTINCT(?cup)) > 10)
ORDER BY ?country
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q414'), ('country', 'Argentina'), ('apparitions', '17')]
[('c', 'http://www.wikidata.org/entity/Q31'), ('country', 'Belgium'), ('apparitions', '13')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('country', 'Brazil'), ('apparitions', '21')]
[('c', 'http://www.wikidata.org/entity/Q21'), ('country', 'England'), ('apparitions', '15')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('apparitions', '15')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('apparitions', '19')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('apparitions', '18')]
[('c', 'http://www.wikidata.org/entity/Q96'), ('country', 'Mexico'), ('apparitions', '16')]
[('c', 'http://www.wikidata.org/entity/Q29'), ('country', 'Spain'), ('apparitions', '15')]
[('c', 'http://www.wikidata.org/entity/Q34'), ('country', 'Sweden'), ('apparitions', '12')]
[('c', 'http://www.wikidata.org/entity/Q39'), ('country', 'Switz

Final query for this task

In [32]:
# write your final query

# Query 45

# task 8: Consider countries who participated at the FIFA World Cup more than 10 times.
#         Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

# Sorting the result is just for make it easier to review.

# wd:Q11571 => Cristiano Ronaldo
# wdt:P1344 => participant in (Query 1) ---> Name of players
# wd:Q27020041 => sports season 
# wd:Q19317 => FIFA World Cup
# wdt:P1532 => country for sport (Query 1)
# wdt:P1923 => participating team (Query 16)
# wd:Q6256 => country
# wdt:P17 => country
# wdt:P3450 => sports season of league or competition (Query 15)


queryString = """
SELECT DISTINCT  ?c ?country (COUNT(DISTINCT(?cup)) as ?apparitions)
WHERE {
    ?cup wdt:P3450 wd:Q19317 ;
         wdt:P1923 ?t.
    ?t   wdt:P1532 ?c.
    
   # get the label
   ?c sc:name ?country.

}
GROUP BY ?c ?country
Having (COUNT(DISTINCT(?cup)) > 10)
ORDER BY DESC(?apparitions)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q155'), ('country', 'Brazil'), ('apparitions', '21')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('apparitions', '19')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('apparitions', '18')]
[('c', 'http://www.wikidata.org/entity/Q414'), ('country', 'Argentina'), ('apparitions', '17')]
[('c', 'http://www.wikidata.org/entity/Q96'), ('country', 'Mexico'), ('apparitions', '16')]
[('c', 'http://www.wikidata.org/entity/Q29'), ('country', 'Spain'), ('apparitions', '15')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('apparitions', '15')]
[('c', 'http://www.wikidata.org/entity/Q21'), ('country', 'England'), ('apparitions', '15')]
[('c', 'http://www.wikidata.org/entity/Q31'), ('country', 'Belgium'), ('apparitions', '13')]
[('c', 'http://www.wikidata.org/entity/Q77'), ('country', 'Uruguay'), ('apparitions', '13')]
[('c', 'http://www.wikidata.org/entity/Q34'), ('country', 'Swed

## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).

In [50]:
# write your queries

# Query 46

# Finding european countries that won the FIFA World Cup at least twice. Beacuse the next part of task is about UEFA Champions League I filtered european countries.

# wdt:P1346 => winner
# wd:Q19317 => FIFA World Cup
# wdt:P3450 => sports season of league or competition
# wdt:P1923 => participating team
# wdt:P1532 => country for sport
# wd:Q6256 => country
# wdt:P30 => continent
# wd:Q46 => Europe
# wd:Q476028 => association football club
# wd:Q18756 => UEFA Champions League


queryString = """
SELECT DISTINCT ?wc ?worldCup ?country
WHERE {

      ?wc wdt:P3450 wd:Q19317 ;
          wdt:P1346 ?t .
      ?t wdt:P1532 ?c .
      ?c wdt:P30 wd:Q46 .
    
   # get the label
    ?c sc:name ?country.
    ?wc sc:name ?worldCup.
}
ORDER BY ?worldCup
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('wc', 'http://www.wikidata.org/entity/Q1477177'), ('worldCup', '1934 FIFA World Cup'), ('country', 'Italy')]
[('wc', 'http://www.wikidata.org/entity/Q131591'), ('worldCup', '1938 FIFA World Cup'), ('country', 'Italy')]
[('wc', 'http://www.wikidata.org/entity/Q131075'), ('worldCup', '1954 FIFA World Cup'), ('country', 'Germany')]
[('wc', 'http://www.wikidata.org/entity/Q134202'), ('worldCup', '1966 FIFA World Cup'), ('country', 'England')]
[('wc', 'http://www.wikidata.org/entity/Q166121'), ('worldCup', '1974 FIFA World Cup'), ('country', 'Germany')]
[('wc', 'http://www.wikidata.org/entity/Q46934'), ('worldCup', '1982 FIFA World Cup'), ('country', 'Italy')]
[('wc', 'http://www.wikidata.org/entity/Q132529'), ('worldCup', '1990 FIFA World Cup'), ('country', 'Germany')]
[('wc', 'http://www.wikidata.org/entity/Q101730'), ('worldCup', '1998 FIFA World Cup'), ('country', 'France')]
[('wc', 'http://www.wikidata.org/entity/Q37285'), ('worldCup', '2006 FIFA World Cup'), ('country', 'Ita

In [58]:
# Query 47

# Filtering european countries won FIFA world cup at least twice. 

# wdt:P1346 => winner
# wd:Q19317 => FIFA World Cup
# wdt:P3450 => sports season of league or competition
# wdt:P1923 => participating team
# wdt:P1532 => country for sport
# wd:Q6256 => country
# wdt:P30 => continent
# wd:Q46 => Europe
# wd:Q476028 => association football club
# wd:Q18756 => UEFA Champions League


queryString = """
SELECT DISTINCT ?c ?country (COUNT(?c) as ?won)
WHERE {

      ?wc wdt:P3450 wd:Q19317 ;
          wdt:P1346 ?t .
      ?t wdt:P1532 ?c .
      ?c wdt:P30 wd:Q46 .
    
   # get the label
    ?c sc:name ?country.
    #?wc sc:name ?worldCup.
}
GROUP BY ?c ?country
HAVING (COUNT(?c) >= 2)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('won', '4')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('won', '4')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('won', '2')]
3


In [85]:
# Query 48

# Task 9: Consider the countries that won the FIFA World Cup at least twice.
#         Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).

# wdt:P1346 => winner
# wd:Q19317 => FIFA World Cup
# wdt:P3450 => sports season of league or competition
# wdt:P1923 => participating team
# wdt:P1532 => country for sport
# wd:Q6256 => country
# wdt:P30 => continent
# wd:Q46 => Europe
# wd:Q476028 => association football club
# wd:Q18756 => UEFA Champions League


queryString = """

SELECT DISTINCT  (?team as ?IRI) ?club 
WHERE {

    ?champLeague wdt:P3450 wd:Q18756 ;
                 wdt:P1346 ?team .
    ?team  wdt:P31 wd:Q476028 ;
           wdt:P17 ?tCountry .      

    # get the label
    ?team sc:name ?club.

    FILTER(?tCountry = ?c)
    ################################################################################################
    # get the country at least 2 times won the FIFA World Cup
        {
            SELECT DISTINCT ?c 
            WHERE {
                ?wc wdt:P3450 wd:Q19317 ;
                wdt:P1346 ?t .
                ?t wdt:P1532 ?c .
                ?c wdt:P30 wd:Q46 .
            }
            GROUP BY ?c ?country
            HAVING (COUNT(?c) >= 2)
        }
    ################################################################################################
}
ORDER BY ASC(?club)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q1543'), ('club', 'A.C. Milan')]
[('IRI', 'http://www.wikidata.org/entity/Q41420'), ('club', 'Borussia Dortmund')]
[('IRI', 'http://www.wikidata.org/entity/Q631'), ('club', 'FC Internazionale Milano')]
[('IRI', 'http://www.wikidata.org/entity/Q51974'), ('club', 'Hamburger SV')]
[('IRI', 'http://www.wikidata.org/entity/Q1422'), ('club', 'Juventus F.C.')]
[('IRI', 'http://www.wikidata.org/entity/Q132885'), ('club', 'Olympique de Marseille')]
6


Final query for this task

In [86]:
# write your final query

# Query 50

# Task 9: Consider the countries that won the FIFA World Cup at least twice.
#         Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).

# wdt:P1346 => winner
# wd:Q19317 => FIFA World Cup
# wdt:P3450 => sports season of league or competition
# wdt:P1923 => participating team
# wdt:P1532 => country for sport
# wd:Q6256 => country
# wdt:P30 => continent
# wd:Q46 => Europe
# wd:Q476028 => association football club
# wd:Q18756 => UEFA Champions League


queryString = """

SELECT DISTINCT  (?team as ?IRI) ?club 
WHERE {

    ?champLeague wdt:P3450 wd:Q18756 ;
                 wdt:P1346 ?team .
    ?team  wdt:P31 wd:Q476028 ;
           wdt:P17 ?tCountry .      

    # get the label
    ?team sc:name ?club.

    FILTER(?tCountry = ?c)
    ################################################################################################
    # get the country at least 2 times won the FIFA World Cup
        {
            SELECT DISTINCT ?c 
            WHERE {
                ?wc wdt:P3450 wd:Q19317 ;
                wdt:P1346 ?t .
                ?t wdt:P1532 ?c .
                ?c wdt:P30 wd:Q46 .
            }
            GROUP BY ?c ?country
            HAVING (COUNT(?c) >= 2)
        }
    ################################################################################################
}
ORDER BY ASC(?club)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q1543'), ('club', 'A.C. Milan')]
[('IRI', 'http://www.wikidata.org/entity/Q41420'), ('club', 'Borussia Dortmund')]
[('IRI', 'http://www.wikidata.org/entity/Q631'), ('club', 'FC Internazionale Milano')]
[('IRI', 'http://www.wikidata.org/entity/Q51974'), ('club', 'Hamburger SV')]
[('IRI', 'http://www.wikidata.org/entity/Q1422'), ('club', 'Juventus F.C.')]
[('IRI', 'http://www.wikidata.org/entity/Q132885'), ('club', 'Olympique de Marseille')]
6
